## Local avoidance with timer

In [1]:
from Thymio import Thymio
import numpy as np
from threading import Timer
import time

# import Move_Thymio as mvt

In [2]:
th = Thymio.serial(port="\\.\COM4", refreshing_rate=0.1)

In [3]:
vreal_max = 0.1525
def vThymio2vReal(vThymio):
    """
    Input: vThymio = 0 - 500
    Output: vReal = 0 - 0.14 m/s
    """
    
    if vThymio > 500:
        print("Warning: vThymio above 500, saturating vReal to 0.14")
        vReal = vreal_max

    elif vThymio < 0:
        print("Warning: vThymio negative")
        vThymio = 0
    
    else:
        vReal = vThymio*vreal_max/500
    
    return vReal 

def vReal2vThymio(vReal):
    """
    Input: vReal = 0 - 0.14 m/s
    Output: vThymio = 0 - 500
    """
    if vReal > vreal_max:
        print("Warning: vReal above 0.14 m/s, saturating vThymio to 500")
        vThymio = 500

    elif vReal < 0:
        print("Warning: vReal negative")
        vThymio = 0
        
    else:
        vThymio = vReal*500/vreal_max
       
        
    return vThymio

def stop():
    
 th.set_var("motor.left.target", 0)
 th.set_var("motor.right.target", 0)
    
def Thymio_rotate(angle, vThymio = 150):

    vReal = vThymio2vReal(vThymio)
    cal_R = 10
    cal_L = 0

    dist_wheel = 95*1e-3 # dist between wheels [m]
    dt = abs(angle)*vReal/(dist_wheel/2)

    if angle > 0:
        th.set_var("motor.left.target", 2**16-(vThymio + cal_L))
        th.set_var("motor.right.target", vThymio + cal_R)
    elif angle < 0:
        th.set_var("motor.left.target", vThymio + cal_L)
        th.set_var("motor.right.target", 2**16-(vThymio + cal_R))
    elif angle == 0:
        stop()
        disp("Warning: angle = 0")


    time.sleep(dt)
    stop()
    
    return dt

def Thymio_translate(d, vThymio = 150):
    """
    Input: d = distance to travel [m], vThymio = 150 (default)
    Output: Thymio moves (default = forward) through a distance d at speed vThymio 
    """
    r_wheel = 43/2*1e-3 #radius of a wheel [m]
    cal_R = 6
    cal_L = 0
    
    vReal = vThymio2vReal(vThymio)
    dt = abs(d)/vReal
    if d > 0: #moves forward
        th.set_var("motor.left.target", vThymio + cal_L)
        th.set_var("motor.right.target", vThymio + cal_R)
        
    elif d < 0: #moves backward
        th.set_var("motor.left.target", 2**16-(vThymio + cal_L))
        th.set_var("motor.right.target", 2**16-(vThymio + cal_R))
        
    elif d == 0:
        stop()
        disp("Warning: dist = 0")
    
    time.sleep(dt)
    stop()
    return dt

In [5]:
def local_avoidance():

    W=np.matrix([[1, 1, -2, 0.5, -0.5], 
                    [-0.5, 0.5, -1, 1,1]])


#     memory=np.array([0,0])
    x=th["prox.horizontal"][0:5]
    
    while np.all(x==0) == False:
        
        x=th["prox.horizontal"][0:5]
        
        y=W.dot(x)
        y=np.divide(y,10)

        #Saturation condition to avoid "bytes must be in range(0, 256)" error

        vL=int(np.round(y[0,0])) 

        if vL > 500:
            vL=500

        vR=int(np.round(y[0,1]))

        if vR > 500:
            vR=500

    #     print('vL = ',vL, 'vR = ',vR)
        memory=[vL,vR]

        if (vL < 0): #convert negative values for Thymio.py
                th.set_var("motor.left.target", 2**16-abs(vL))
        else:
                th.set_var("motor.left.target", vL)

        if (vR < 0):
                th.set_var("motor.right.target", 2**16-abs(vR))
        else:
                th.set_var("motor.right.target", vR)



#         elif(0==(memory[0] and memory[1])):

#             th.set_var("motor.left.target", 100)
#             th.set_var("motor.right.target", 100)


#         else:

#             if (memory[0]==0 and memory[1]!=0):
#                 th.set_var("motor.left.target", memory[1])
#                 th.set_var("motor.right.target", memory[1])

#             elif (memory[1]==0 and memory[0]!=0):
#                 th.set_var("motor.left.target", memory[0])
#                 th.set_var("motor.right.target", memory[0])

In [6]:
class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [12]:
Ts = 0.2

local_avoid = False

acquire_data = True
prox_sensors = []

def get_data(local_avoid): #Function to be repeated with timer
    """
    Checks at regular intervals the prox sensor and enters local avoidance if obstacle detected
    """
    
    prox_sensors = th["prox.horizontal"][0:5]
    print(prox_sensors)
    
    if np.all(prox_sensors == 0) == False and not local_avoid:
        
        print("entering local avoid")
        local_avoid = True
        local_avoidance()
        local_avoid = False
        
    
rt = RepeatedTimer(Ts, get_data,[local_avoid]) # it auto-starts, no need of rt.start()

d = 0.20 # m
Thymio_translate(d)
print("bonjour")

rt.stop()

[2064, 1626, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
[0, 0, 0, 1660, 1230]
bonjour


In [13]:
stop()